In [49]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from helpers import *
from implementations import *
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
DATA_TRAIN_PATH = 'train.csv'
y_or, tX_or, ids = load_csv_data(DATA_TRAIN_PATH)

In [51]:
y = y_or
tX = tX_or

In [52]:
tX = pd.DataFrame(tX)
tX.where(tX!=-999, inplace=True)
tX.fillna(tX.median(), inplace=True)    
tX = tX.values

In [53]:
y = y_for_logistic(y)

In [54]:
x_tr_tot, x_te_tot, y_tr_tot, y_te_tot = split_data(y,tX,0.8,1)

In [55]:
initial_w = np.zeros(x_tr_tot.shape[1])
max_iters = 10000
gamma = 1e-10
w, loss = logistic_regression(y_tr_tot, x_tr_tot, initial_w, max_iters, gamma)

Current iteration=0, loss=138629.43611198905
Current iteration=1000, loss=106751.60151235267
Current iteration=2000, loss=106408.02717390667
Current iteration=3000, loss=106299.1933633236
Current iteration=4000, loss=106253.50135402067
Current iteration=5000, loss=106230.50607151032
Current iteration=6000, loss=106216.27284014336
Current iteration=7000, loss=106205.50922362538
Current iteration=8000, loss=106196.14474365817
Current iteration=9000, loss=106187.35953373656


In [58]:
y_pred = predict_labels(w, x_te_tot, True)
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.7362


In [59]:
DATA_TEST_PATH = 'test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [60]:
#cell for logistic
y_pred_test = predict_labels(w, tX_test, True)
OUTPUT_PATH = 'submission.csv'
create_csv_submission(ids_test, y_pred_test, OUTPUT_PATH, logistic=True) #logistic flags convert [0, 1] to [-1, 1]

In [61]:
initial_w = np.zeros(x_tr_tot.shape[1])
lambda_ = 1e-8
max_iters = 10000
gamma = 1e-10
w, loss = reg_logistic_regression(y_tr_tot, x_tr_tot, lambda_, initial_w, max_iters, gamma)

Current iteration=0, loss=134001.19392646643
Current iteration=1000, loss=106750.92508967363
Current iteration=2000, loss=106407.85577669005
Current iteration=3000, loss=106299.12698797951
Current iteration=4000, loss=106253.47056667147
Current iteration=5000, loss=106230.48878656604
Current iteration=6000, loss=106216.26085869093
Current iteration=7000, loss=106205.49936369108
Current iteration=8000, loss=106196.13575003046
Current iteration=9000, loss=106187.3509072507


In [62]:
y_pred = predict_labels(w, x_te_tot, True)
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.7362


In [63]:
#cell for logistic
y_pred_test = predict_labels(w, tX_test, True)
OUTPUT_PATH = 'submission.csv'
create_csv_submission(ids_test, y_pred_test, OUTPUT_PATH, logistic=True) #logistic flags convert [0, 1] to [-1, 1]

In [15]:
def cross_validation(y, x, k_indices, k, lambda_, gamma, acc = False, ls = False):
    """return the loss of ridge regression."""
    losses_tr = []
    losses_te = []
    accuracies = []
    ws = []
    initial_w = np.zeros(x.shape[1])

    
    for k_group in range(k):
        index_te = k_indices[k_group]
        index_tr = np.setdiff1d(np.arange(len(y)), index_te)
        #print(index_te, index_tr)
        x_te = x[index_te]
        x_tr = x[index_tr]
        y_te = y[index_te]
        y_tr = y[index_tr]
        
        # form data with polynomial degree
        x_te_poly = x_te
        x_tr_poly = x_tr
        
        if (ls==False):
            w, _ = reg_logistic_regression(y_tr, x_tr_poly, lambda_, initial_w, 1000, gamma)
        else:
            w = least_squares_lstsq_ver(y_tr, x_tr_poly)
        
        # calculate the loss for train and test data
        rmse_tr = calculate_loss(y_tr, x_tr_poly, w)
        rmse_te = calculate_loss(y_te, x_te_poly, w)
        #print(lambda_, rmse_te)
        losses_tr.append(rmse_tr)
        losses_te.append(rmse_te)
        
        y_pred = predict_labels(w, x_te_poly)
        accuracies.append(accuracy(y_te, y_pred))
        
        ws.append(w)
        
    if acc==False:
        loss_tr = np.median(losses_tr)
        loss_te = np.median(losses_te)
        return loss_tr, loss_te, np.mean(ws, axis=0)
    else:
        return np.mean(accuracies), np.mean(ws, axis=0)

In [11]:
def select_best_hypers(y, x, k_fold, seed=1):
    #y_sub, x_sub = get_subsample(y, x, 100000, seed)
    lambdas = np.logspace(-8, 0, 8)
    gammas = np.logspace(-6, 0, 5)
    #lambdas=[1e-06]
    k_indices = build_k_indices(y, k_fold, seed)
    loss_min = np.inf
    gamma_star = 0
    lambda_star = 0
    w_star = 0
    for gamma in gammas:
        for lambda_ in lambdas:
            #print(degree)
            loss_tr, loss_te, w = cross_validation(y, x, k_indices, k_fold, lambda_, gamma)
            #print(loss_te)
            if loss_te < loss_min:
                loss_min = loss_te
                print("New loss: {}, gamma: {}, lambda: {}".format(loss_te, gamma, lambda_))
                gamma_star = gamma
                lambda_star = lambda_
                w_star = w
    return gamma_star, lambda_star, loss_min, w_star

In [20]:
k_fold = 4
gammas_star=[]
lambdas_star=[]
w_star = []
for jet in range(0, 8):
    print("jet {}: ".format(jet))
    gamma_star, lambda_star, loss, w = select_best_hypers(y_tr[jet], tX_tr[jet], k_fold, 1)
    gammas_star.append(gamma_star)
    lambdas_star.append(lambda_star)
    w_star.append(w)
    #print("jet {}: Best accuracy {}, degree: {},  lambda: {}".format(jet, acc, degree_star, lambda_star))
    print("jet {}: best loss: {}, degree: {}, lambda: {}".format(jet, loss, gamma_star, lambda_star))

jet 0: 
New loss: 1223.9791281210773, gamma: 1e-06, lambda: 1e-08
New loss: 936.8265655823413, gamma: 3.1622776601683795e-05, lambda: 1e-08
jet 0: best loss: 936.8265655823413, degree: 3.1622776601683795e-05, lambda: 1e-08
jet 1: 
New loss: 8487.580246188481, gamma: 1e-06, lambda: 1e-08
jet 1: best loss: 8487.580246188481, degree: 1e-06, lambda: 1e-08
jet 2: 
New loss: 476.60760865688735, gamma: 1e-06, lambda: 1e-08
New loss: 418.492834146361, gamma: 3.1622776601683795e-05, lambda: 1e-08
jet 2: best loss: 418.492834146361, degree: 3.1622776601683795e-05, lambda: 1e-08
jet 3: 
New loss: 8371.862932929653, gamma: 1e-06, lambda: 1e-08
jet 3: best loss: 8371.862932929653, degree: 1e-06, lambda: 1e-08
jet 4: 
New loss: 253.43329536578767, gamma: 1e-06, lambda: 1e-08
New loss: 246.94184229189779, gamma: 3.1622776601683795e-05, lambda: 1e-08
jet 4: best loss: 246.94184229189779, degree: 3.1622776601683795e-05, lambda: 1e-08
jet 5: 
New loss: 6504.6613494500925, gamma: 1e-06, lambda: 1e-08
jet

In [8]:
y_tr, tX_tr, indexes_tr, means_tr, std_tr = preprocessing(x_tr_tot, y_tr_tot)
y_te, tX_te, indexes_te, means_te, std_te = preprocessing(x_te_tot, y_te_tot)

In [37]:
list_loss = []
list_w = []
for jet in range(0,8):
    initial_w = np.zeros(tX_tr[jet].shape[1])
    w, loss = reg_logistic_regression(y_tr[jet], tX_tr[jet], lambdas_star[jet], initial_w, 10000, gammas_star[jet])
    list_loss.append(loss)
    list_w.append(w)

In [38]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.72788


In [41]:
lambdas_star

[1e-08, 1e-08, 1e-08, 1e-08, 1e-08, 1e-08, 1e-08, 1e-08]

In [31]:
DATA_TEST_PATH = 'test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test_new, indexes_test_new, means_test, stds_test = preprocessing(tX_test) #same function as train,

In [32]:
#cell for logistic
y_pred_test = build_predictions(tX_test_new, indexes_test_new, list_w, logistic=True)
OUTPUT_PATH = 'submission.csv'
create_csv_submission(ids_test, y_pred_test, OUTPUT_PATH, logistic=True) #logistic flags convert [0, 1] to [-1, 1]